In [ ]:
%matplotlib inline
import pandas as pd

train = pd.read_csv("./data.csv")
train.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(60, 9))
sns.countplot('deviceid', data=train)

In [ ]:
sns.countplot(x='subid', data=train)

In [ ]:
sns.countplot(x='type', data=train)
#%more neural_network.py

In [ ]:
import numpy as np
%matplotlib inline
from neural_network import NeuralNetwork
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt


# 读取文件
fr = open("./data.csv")
data_mat = []; label_mat = []
dict_time = {}; fr.readline()
# 读取当前行
for line in fr.readlines()[4:]:
    cur_line = line.strip().split(',')
    time = cur_line[-1]; type = cur_line[3]
    if time in dict_time:
        dict_time[time][type] = cur_line[-2]
    else:
        dict_time[time] = {type: cur_line[-2]}
# 生成新数据集
x_arr = []
for key in sorted(dict_time.keys()):
    type_value = dict_time[key]
    if '"bat"' not in type_value and len(type_value) == 4:
        type_value_arr= [float(type_value['"temp"']), float(type_value['"co2"']), float(type_value['"hum"']), float(type_value['"lx"'])]
        x_arr.append(type_value_arr)

nn = NeuralNetwork([4,20,1], 'logistic')
x_arr = np.array(x_arr)
# data normalizate
x_arr = np.where(x_arr>0, x_arr, np.NAN)
mean_x = np.nanmean(x_arr, 0)
x_arr = np.where(x_arr>0, x_arr, mean_x)
x_min_max = np.max(x_arr, 0)-np.min(x_arr,0)
x_arr = (x_arr-np.min(x_arr,0))/x_min_max
y_arr = x_arr[:,0]
# feature select
## Maximum information based non parametric exploration
from minepy import MINE
m = MINE()
print "t时刻温度与t-1、t－2、t－3、t－4时刻的温度的MIC"
m.compute_score(x_arr[:-1,0],y_arr[1:])
print m.mic()
m.compute_score(x_arr[:-2,0],y_arr[2:])
print m.mic()
m.compute_score(x_arr[:-3,0],y_arr[3:])
print m.mic()
m.compute_score(x_arr[:-4,0],y_arr[4:])
print m.mic()
print "t时刻温度与t-1、t－2、t－3、t－4时刻的二氧化碳浓度的MIC"
m.compute_score(x_arr[:-1,1],y_arr[1:])
print m.mic()
m.compute_score(x_arr[:-2,1],y_arr[2:])
print m.mic()
m.compute_score(x_arr[:-3,1],y_arr[3:])
print m.mic()
m.compute_score(x_arr[:-4,1],y_arr[4:])
print m.mic()
print "t时刻温度与t-1、t－2、t－3、t－4时刻的湿度的MIC"
m.compute_score(x_arr[:-1,2],y_arr[1:])
print m.mic()
m.compute_score(x_arr[:-2,2],y_arr[2:])
print m.mic()
m.compute_score(x_arr[:-3,2],y_arr[3:])
print m.mic()
m.compute_score(x_arr[:-4,2],y_arr[4:])
print m.mic()
print "t时刻温度与t-1、t－2、t－3、t－4时刻的光照的MIC"
m.compute_score(x_arr[:-1,3],y_arr[1:])
print m.mic()
m.compute_score(x_arr[:-2,3],y_arr[2:])
print m.mic()
m.compute_score(x_arr[:-3,3],y_arr[3:])
print m.mic()
m.compute_score(x_arr[:-4,3],y_arr[4:])
print m.mic()


In [ ]:
nn = NeuralNetwork([4,20,1], 'logistic')
x_train, x_test, y_train, y_test = train_test_split(x_arr[:-1], y_arr[1:])
print "start fitting"
nn.fit(x_train, y_train)
errors = [];times = []
for i in xrange(x_test.shape[0]):
    o = nn.predict(x_test[i])
    errors.append((y_test[i]-o)*x_min_max[0])
    times.append(i)
plt.plot(times, errors)
plt.show()

In [ ]:
import numpy as np
%matplotlib inline
import seaborn as sns
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt


# 读取文件
fr = open("./data.csv")
data_mat = []; label_mat = []
dict_time = {}; fr.readline()
# 读取当前行
for line in fr.readlines()[4:12313]:
    cur_line = line.strip().split(',')
    time = cur_line[-1]; type = cur_line[3]
    if time in dict_time:
        dict_time[time][type] = cur_line[-2]
    else:
        dict_time[time] = {type: cur_line[-2]}
# 生成新数据集
x_arr = []
for key in sorted(dict_time.keys()):
    type_value = dict_time[key]
    if len(type_value) == 4:
        type_value_arr= [float(type_value['"temp"']), float(type_value['"co2"']), float(type_value['"hum"']), float(type_value['"lx"'])]
        x_arr.append(type_value_arr)

x_arr = np.array(x_arr); y_arr = np.array(y_arr)
# data normalizate
x_arr = np.where(x_arr>0, x_arr, np.NAN)
mean_x = np.nanmean(x_arr, 0)
x_arr = np.where(x_arr>0, x_arr, mean_x)
x_min_max = np.max(x_arr, 0)-np.min(x_arr,0)
x_arr = (x_arr-np.min(x_arr,0))/x_min_max
y_arr = x_arr[:,0]

x_train, x_test, y_train, y_test = train_test_split(x_arr[:-1], y_arr[1:])

print "start training"
xgb_model = xgb.XGBRegressor().fit(x_train, y_train)

# this is prediction
predictions = xgb_model.predict(x_test)
errors = [];times = []
for i in xrange(x_test.shape[0]):
    errors.append((y_test[i]-predictions[i])*x_min_max[0])
    times.append(i)
plt.plot(times, errors)
plt.show()